In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS, GLSAR
from statsmodels.stats.stattools import durbin_watson
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8-darkgrid')

print('Libraries loaded')

Libraries loaded


In [2]:
# Load data
df = pd.read_csv('../data/raw/df_eurogreen_japan_complete.csv')
print(f'Dataset: {df.shape[0]} years, {df.shape[1]} variables')
print(f'Period: {int(df["Year"].min())}-{int(df["Year"].max())}')

# Create derived variables
df['GDP'] = (df['Priv_Cons'] + df['Gov_Cons'] + df['Corp_Equip_Inv'] + 
             df['Public_Inv'] + df['Exports'] - df['Imports'])

df['Real_GDP'] = (df['Real_Priv_Cons'] + df['Real_Gov_Cons'] + df['Real_Corp_Equip_Inv'] + 
                  df['Real_Public_Inv'] + df['Real_Exports'] - df['Real_Imports'])

# National income (for consumption function)
df['National_Income'] = df['GDP'] - df['G_Taxes_Production']

# Investment variables
df['Profit_Share'] = df['F_Op_Surplus'] / df['GDP']
df['Investment_GDP'] = df['Corp_Equip_Inv'] / df['GDP']
df['Capacity_Util'] = df['Real_GDP'] / df['Real_GDP'].rolling(5, center=True, min_periods=3).mean()

# Phillips curve variables
df['Wage_Rate'] = df['H_Comp_Employees'] / df['L_Employed']
df['Wage_Growth'] = df['Wage_Rate'].pct_change() * 100
df['Inflation'] = df['Def_Priv_Cons'].pct_change() * 100

# Gini variables
df['Capital_Share'] = df['F_Op_Surplus'] / df['GDP']

print('\nVariables created')

Dataset: 30 years, 93 variables
Period: 1994-2023

Variables created


In [3]:
# Prepare data
cons_data = pd.DataFrame({
    'C': df['Priv_Cons'],
    'YN': df['National_Income']
}).dropna()

# OLS estimation
y = cons_data['C'].values
X = sm.add_constant(cons_data[['YN']]).values

# AR(1) correction for autocorrelation
model_cons = GLSAR(y, X, rho=1)
results_cons = model_cons.iterative_fit(maxiter=10)

print('='*80)
print('CONSUMPTION FUNCTION')
print('='*80)
print(results_cons.summary())

alpha_0 = results_cons.params[0]
alpha_1 = results_cons.params[1]
alpha_2 = 0.02  # Wealth effect from literature

print(f'\nα₀ (autonomous): {alpha_0:,.2f}')
print(f'α₁ (MPC): {alpha_1:.4f}')
print(f'α₂ (wealth): {alpha_2:.4f}')

CONSUMPTION FUNCTION
                           GLSAR Regression Results                           
Dep. Variable:                      y   R-squared:                       0.601
Model:                          GLSAR   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     40.71
Date:                Sun, 04 Jan 2026   Prob (F-statistic):           7.81e-07
Time:                        14:25:35   Log-Likelihood:                -277.45
No. Observations:                  29   AIC:                             558.9
Df Residuals:                      27   BIC:                             561.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.597e+04   3.37e+0

In [4]:
# Prepare data
inv_data = pd.DataFrame({
    'I_share': df['Investment_GDP'],
    'profit_lag': df['Profit_Share'].shift(1),
    'capacity': df['Capacity_Util']
}).dropna()

# AR(1) estimation
y = inv_data['I_share'].values
X = sm.add_constant(inv_data[['profit_lag', 'capacity']]).values

model_inv = GLSAR(y, X, rho=1)
results_inv = model_inv.iterative_fit(maxiter=10)

print('='*80)
print('INVESTMENT FUNCTION')
print('='*80)
print(results_inv.summary())

beta_0 = results_inv.params[0]
beta_1 = results_inv.params[1]
beta_2 = results_inv.params[2]

print(f'\nβ₀: {beta_0:.6f}')
print(f'β₁ (profit): {beta_1:.6f}')
print(f'β₂ (accelerator): {beta_2:.6f}')

INVESTMENT FUNCTION
                           GLSAR Regression Results                           
Dep. Variable:                      y   R-squared:                       0.348
Model:                          GLSAR   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     6.664
Date:                Sun, 04 Jan 2026   Prob (F-statistic):            0.00479
Time:                        14:26:52   Log-Likelihood:                 109.37
No. Observations:                  28   AIC:                            -212.7
Df Residuals:                      25   BIC:                            -208.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0528      0.077

In [5]:
# Prepare data
phillips_data = pd.DataFrame({
    'wage_growth': df['Wage_Growth'],
    'unemployment': df['L_Unemployment_Rate'],
    'inflation': df['Inflation']
}).dropna()

# OLS estimation
y = phillips_data['wage_growth']
X = sm.add_constant(phillips_data[['unemployment', 'inflation']])

model_phillips = OLS(y, X).fit()

print('='*80)
print('PHILLIPS CURVE')
print('='*80)
print(model_phillips.summary())

gamma_0 = model_phillips.params['const']
gamma_1 = model_phillips.params['unemployment']
gamma_2 = model_phillips.params['inflation']

print(f'\nγ₀: {gamma_0:.4f}')
print(f'γ₁ (unemployment): {gamma_1:.4f}')
print(f'γ₂ (inflation): {gamma_2:.4f}')

PHILLIPS CURVE
                            OLS Regression Results                            
Dep. Variable:            wage_growth   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     18.75
Date:                Sun, 04 Jan 2026   Prob (F-statistic):           9.09e-06
Time:                        14:27:42   Log-Likelihood:                -35.271
No. Observations:                  29   AIC:                             76.54
Df Residuals:                      26   BIC:                             80.64
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            2.6810      1.04

In [6]:
# Use original Gini data only (no interpolation)
gini_data = pd.DataFrame({
    'Gini': df['Gini'],
    'Capital_Share': df['Capital_Share']
}).dropna()

print(f'Gini observations: n={len(gini_data)}')

# OLS estimation
y = gini_data['Gini']
X = sm.add_constant(gini_data[['Capital_Share']])

model_gini = OLS(y, X).fit()

print('='*80)
print('GINI EQUATION')
print('='*80)
print(model_gini.summary())

delta_0 = model_gini.params['const']
delta_1 = model_gini.params['Capital_Share']

print(f'\nδ₀: {delta_0:.4f}')
print(f'δ₁ (capital share): {delta_1:.4f}')
print(f'p-value: {model_gini.pvalues["Capital_Share"]:.4f}')
print('\nNote: Not statistically significant due to small sample size')

Gini observations: n=9
GINI EQUATION
                            OLS Regression Results                            
Dep. Variable:                   Gini   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1.403
Date:                Sun, 04 Jan 2026   Prob (F-statistic):              0.275
Time:                        14:28:32   Log-Likelihood:                 22.912
No. Observations:                   9   AIC:                            -41.82
Df Residuals:                       7   BIC:                            -41.43
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const    

In [7]:
# Use 2019-2023 average
recent = df.iloc[-5:]

# Tax rates
tau_income = (recent['H_Direct_Taxes'] / recent['H_Comp_Employees']).mean()
tau_corporate = (recent['F_Corp_Taxes'] / recent['F_Op_Surplus']).mean()
tau_production = (recent['G_Taxes_Production'] / df['GDP'].iloc[-5:]).mean()

# Social security
ss_contrib_rate = (recent['H_Social_Contrib'] / recent['H_Comp_Employees']).mean()
ss_benefit_rate = (recent['H_Social_Benefits'] / recent['H_Comp_Employees']).mean()

# Interest rates
r_deposits = (recent['B_Interest_Paid'] / recent['B_Deposits_Liability']).mean()
r_loans = (recent['B_Interest_Rec'] / recent['B_Loans_Asset']).mean()
r_bonds = (recent['G_Interest_Paid'] / recent['G_Debt_Securities']).mean()

# Depreciation
delta_k = 0.05

print('CALIBRATED PARAMETERS (2019-2023 average)')
print('='*80)
print(f'Tax rates:')
print(f'  Income:      {tau_income:.4f}')
print(f'  Corporate:   {tau_corporate:.4f}')
print(f'  Production:  {tau_production:.4f}')
print(f'\nSocial security:')
print(f'  Contribution: {ss_contrib_rate:.4f}')
print(f'  Benefits:     {ss_benefit_rate:.4f}')
print(f'\nInterest rates:')
print(f'  Deposits:    {r_deposits:.4f}')
print(f'  Loans:       {r_loans:.4f}')
print(f'  Bonds:       {r_bonds:.4f}')
print(f'\nDepreciation:  {delta_k:.4f}')

CALIBRATED PARAMETERS (2019-2023 average)
Tax rates:
  Income:      0.1100
  Corporate:   0.5728
  Production:  0.0937

Social security:
  Contribution: 0.2953
  Benefits:     0.2810

Interest rates:
  Deposits:    0.0113
  Loans:       0.0186
  Bonds:       0.0068

Depreciation:  0.0500


In [8]:
# Compile all parameters
parameters = {
    'consumption': {
        'alpha_0': float(alpha_0),
        'alpha_1': float(alpha_1),
        'alpha_2': float(alpha_2)
    },
    'investment': {
        'beta_0': float(beta_0),
        'beta_1': float(beta_1),
        'beta_2': float(beta_2)
    },
    'phillips': {
        'gamma_0': float(gamma_0),
        'gamma_1': float(gamma_1),
        'gamma_2': float(gamma_2)
    },
    'gini': {
        'delta_0': float(delta_0),
        'delta_1': float(delta_1),
        'n_obs': 9,
        'note': 'Small sample, p=0.275'
    },
    'calibrated': {
        'tau_income': float(tau_income),
        'tau_corporate': float(tau_corporate),
        'tau_production': float(tau_production),
        'ss_contrib_rate': float(ss_contrib_rate),
        'ss_benefit_rate': float(ss_benefit_rate),
        'r_deposits': float(r_deposits),
        'r_loans': float(r_loans),
        'r_bonds': float(r_bonds),
        'delta_k': float(delta_k)
    }
}

# Save to JSON
output_dir = Path('../results')
output_dir.mkdir(exist_ok=True)

with open(output_dir / 'parameters.json', 'w') as f:
    json.dump(parameters, f, indent=2)

print('✓ Parameters saved to results/parameters.json')
print('\nEstimated: 13 behavioral parameters')
print('Calibrated: 9 institutional parameters')
print('Total: 22 parameters')

✓ Parameters saved to results/parameters.json

Estimated: 13 behavioral parameters
Calibrated: 9 institutional parameters
Total: 22 parameters
